<a id="top"></a>
# NDVI Phenology
<hr>

# Notebook Summary

* LANDSAT 7, LANDSAT 8, or both are used to detect changes in plant life over time.

<hr>

# Algorithmic process  

* [Import Dependencies and Connect to the Data Cube](#import)
* [Choose Platform and Product](#plat_prod)
* [Get the Maximum Extents of the Cube](#extents)
* [Define the Extents of the Analysis](#define_extents) (selecting too much can make the acquisition process slow)
* [Load Data from the Data Cube](#retrieve_data)
* [Calculate NDVI](#calculate)
* [Examine the Selected Area](#area_analysis)
* [Create a Max NDVI Composite](#mosaic)
* [Export the Mosaic to a PNG and a GeoTIFF](#export)

<hr>

# How It Works

To detect changes in plant life, we use a measure called NDVI. 
* <font color=green>NDVI</font> is the ratio of the difference between the amount of near infrared light <font color=red>(NIR)</font> and red light <font color=red>(RED)</font> divided by their sum.
<br>

$$ NDVI =  \frac{(NIR - RED)}{(NIR + RED)}$$  

<br>
<div class="alert-info">
The intention is to observe how much red light is being absorbed versus reflected. Photosynthetic plants absorb most of the visible spectrum's wavelengths when they are healthy. When they aren't healthy, more of that light will get reflected.  This makes the difference between <font color=red>NIR</font> and <font color=red>RED</font> much smaller, which will lower the <font color=green>NDVI</font>.  The resulting values from doing this over several pixels can be used to create visualizations for the changes in the amount of photosynthetic vegetation in large areas.
</div>

## <span id="import">Import Dependencies and Connect to the Data Cube [&#9652;](#top)</span>

In [ ]:
%matplotlib inline
import warnings
# Supress Warning 
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

# Import the datacube and the API
import datacube
from utils.data_cube_utilities.data_access_api import DataAccessApi
import numpy as np
import pandas as pd
import xarray as xr
import datetime as dt

In [ ]:
# Create an instance of the datacube and API.
api = DataAccessApi(config="/home/localuser/.datacube.conf")
dc = api.dc

## <span id="plat_prod">Choose Platform and Product [&#9652;](#top)</span>

In [ ]:
# Get available products
products_info = dc.list_products()

In [ ]:
# List LANDSAT 7 products
print("LANDSAT 7 Products:")
products_info[["platform", "name"]][products_info.platform == "LANDSAT_7"]

In [ ]:
# List LANDSAT 8 products
print("LANDSAT 8 Products:")
products_info[["platform", "name"]][products_info.platform == "LANDSAT_8"]

In [ ]:
# These are the platforms (satellites) and products (datacube sets) 
# used for this demonstration.
use_Landsat7 = True
use_Landsat8 = True
platforms = []
products = []
if use_Landsat7:
    platforms.append('LANDSAT_7')
    products.append('ls7_ledaps_ghana')
if use_Landsat8:
    platforms.append('LANDSAT_8')
    products.append('ls8_lasrc_ghana')

## <span id="extents">Get the Maximum Extents of the Cube [&#9652;](#top)</span>

In [ ]:
for product, platform in zip(products, platforms):
    prod_extents = api.get_query_metadata(platform=platform, product=product, measurements=[])

    latitude_extents = prod_extents['lat_extents']
    longitude_extents = prod_extents['lon_extents']
    time_extents = list(map(lambda time: time.strftime('%Y-%m-%d'), prod_extents['time_extents']))

    print("{}:".format(platform))
    print("Lat bounds:", latitude_extents)
    print("Lon bounds:", longitude_extents)
    print("Time bounds:", time_extents)

In [ ]:
from utils.data_cube_utilities.dc_load import get_overlapping_area
from utils.data_cube_utilities.dc_display_map import display_map

full_lat, full_lon, min_max_dates = get_overlapping_area(api, platforms, products)

# Display the total shared area available for these datacube products.
display_map(latitude = full_lat,longitude = full_lon)

## <span id="define_extents">Define the Extents of the Analysis [&#9652;](#top)</span>

**Specify start and end dates in the same order as platforms and products**

In [ ]:
# Use these four lines to select the time slice common to all products.
# min_start_date_mutual = np.max(min_max_dates[:,0])
# max_end_date_mutual = np.min(min_max_dates[:,1])
# start_dates = [min_start_date_mutual, min_start_date_mutual]
# end_dates = [max_end_date_mutual, max_end_date_mutual]
# Use these two lines to select all data available to each product.
# start_dates = min_max_dates[:,0]
# end_dates = min_max_dates[:,1]

# Select a subset of the time available.
start_date, end_date = dt.datetime(2016,1,1), dt.datetime(2017,12,1)
# start_date, end_date = dt.datetime(2014,9,1), dt.datetime(2015,3,1)

**Specify an area to analyze**

In [ ]:
# Specify latitude and longitude bounds of an interesting area within the full extents

# Ghana
# min_lat_small, max_lat_small = (5.7261, 5.7390) # Crops NW of Accra (small)
# min_lon_small, max_lon_small = (-0.4960, -0.4889) # Crops NW of Accra (small)
# min_lat_small, max_lat_small = (5.7259, 5.7517) # Crops NW of Accra (medium)
# min_lon_small, max_lon_small = (-0.5308, -0.5143) # Crops NW of Accra (medium)
min_lat_small, max_lat_small = (8.0074, 8.0203) # Central Ghana - West of Kintampo
min_lon_small, max_lon_small = (-2.0486, -2.0332) # Central Ghana - West of Kintampo
    
# Vietnam
# min_lat_small, max_lat_small = (10.95, 11.00) # Area #1
# min_lon_small, max_lon_small = (107.15, 107.20) # Area #1
# min_lat_small, max_lat_small = (11.10, 11.39) # Area #2
# min_lon_small, max_lon_small = (106.8, 106.92) # Area #2
# min_lat_small, max_lat_small = (9.8, 10.0) # Area #3
# min_lon_small, max_lon_small = (105.2, 105.4) # Area #3

**Visualize the selected area**

In [ ]:
lon_small = (min_lon_small, max_lon_small)
lat_small = (min_lat_small, max_lat_small)
display_map(lat_small, lon_small)

## <span id="retrieve_data">Load Data from the Data Cube [&#9652;](#top)</span>

In [ ]:
from utils.data_cube_utilities.dc_load import load_multiplatform

measurements = ['red', 'blue', 'green', 'nir', 'swir1', 'swir2', 'pixel_qa']
dataset, clean_mask, _ = \
    load_multiplatform(dc, platforms, products, 
                       load_params=dict(lat=lat_small, lon=lon_small, time=(start_date, end_date),
                                        measurements=measurements))
cleaned_dataset = dataset.where(clean_mask)
del dataset

## <span id='calculate'>Calculate NDVI [&#9652;](#top)</span>

In [ ]:
from utils.data_cube_utilities.dc_ndvi_anomaly import NDVI
ndvi_arr = NDVI(cleaned_dataset)
cleaned_dataset['ndvi'] = ndvi_arr

## <span id="area_analysis">Examine the Selected Area [&#9652;](#top)</span>

**If no plots appear in the figures below, there is no data available for the region selected**

**Box-and-Whisker Plot by Full Time Period, Week, Month, Week of Year, or Month of Year.**

In [ ]:
from utils.data_cube_utilities.plotter_utils import xarray_time_series_plot

# Specify whether to plot a Gaussian curve fit of the mean of NDVI along time.
plot_curve_fit_ndvi_mean = True
max_times_per_plot = 10 # The maximum number of acquisitions with data that appear in each plot.

# Can be one of [None, 'week', 'month', 'weekofyear', 'monthofyear'].
for bin_by in ['monthofyear']:
    aggregated_by_str = None
    if bin_by is None:
        plotting_data = cleaned_dataset
    elif bin_by == 'week':
        plotting_data = cleaned_dataset.resample(time='1w').mean()
        aggregated_by_str = 'Week'
    elif bin_by == 'month':
        plotting_data = cleaned_dataset.resample(time='1m').mean()
        aggregated_by_str = 'Month'
    elif bin_by == 'weekofyear':
        plotting_data = cleaned_dataset.groupby('time.week').mean(dim=('time'))
        aggregated_by_str = 'Week of Year'
    elif bin_by == 'monthofyear':
        plotting_data = cleaned_dataset.groupby('time.month').mean(dim=('time'))
        aggregated_by_str = 'Month of Year'
    
params = dict(dataset=plotting_data, plot_descs={'ndvi':{'none':[
    {'box':{'boxprops':{'facecolor':'forestgreen'}}}]}})
if plot_curve_fit_ndvi_mean:
    params['plot_descs']['ndvi']['mean'] = [{'poly': {'degree': 3}}]
    
params['scale_params'] = 'norm'
    
xarray_time_series_plot(**params, fig_params=dict(figsize=(8,4), dpi=150))
plt.title('Box-and-Whisker Plot by Full Time Period')
print("NDVI {}".format("(Aggregated by {})".format(aggregated_by_str) if aggregated_by_str is not None else ""))

## <span id="mosaic">Create a Max NDVI Composite [&#9652;](#top)</span>

In [ ]:
from utils.data_cube_utilities.dc_mosaic import create_max_ndvi_mosaic
from utils.data_cube_utilities.dc_rgb import rgb
max_ndvi_mosaic = create_max_ndvi_mosaic(cleaned_dataset, clean_mask)
rgb(max_ndvi_mosaic)
plt.show()

## <span id="export">Export the Mosaic to a PNG and a GeoTIFF [&#9652;](#top)</span>

**Export to PNG**

In [ ]:
from utils.data_cube_utilities.dc_utilities import write_png_from_xr
import os
png_dir = 'output/pngs'
if not os.path.exists(png_dir):
    os.makedirs(png_dir)
write_png_from_xr('{}/NDVI_Phenology_Max_NDVI_Mosaic.png'.format(png_dir), max_ndvi_mosaic, ['red','green','blue'], scale=(0,4000))

**Export to GeoTIFF**

In [ ]:
from utils.data_cube_utilities.import_export import export_xarray_to_geotiff
geotiff_dir = 'output/geotiffs/NDVI_Phenology'
if not os.path.exists(geotiff_dir):
    os.makedirs(geotiff_dir)
export_xarray_to_geotiff(cleaned_dataset, "{}/NDVI_Phenology".format(geotiff_dir))